<a href="https://colab.research.google.com/github/vk-tor/bank-credit-analyst/blob/main/LABPROG_An%C3%A1lise_de_Cr%C3%A9dito.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Instalação do Prolog (SWI-Prolog) e ponte para Python
!sudo apt-get install swi-prolog
!pip install pyswip

#Instalação do Lisp (SBCL - Steel Bank Common Lisp)
!sudo apt-get install sbcl

print("Ambiente configurado com sucesso!")

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
swi-prolog is already the newest version (8.4.2+dfsg-2ubuntu1).
0 upgraded, 0 newly installed, 0 to remove and 1 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
sbcl is already the newest version (2:2.1.11-1).
0 upgraded, 0 newly installed, 0 to remove and 1 not upgraded.
Ambiente configurado com sucesso!


In [ ]:
class Cliente:
    def __init__(self, nome, idade, renda, score, negativado):
        self.nome = nome
        self.idade = idade
        self.renda = renda
        self.score = score
        self.negativado = negativado # Booleano: True se tiver nome sujo

    def __str__(self):
        return f"Cliente: {self.nome} | Score: {self.score} | Renda: R${self.renda:.2f}"

class Proposta:
    def __init__(self, valor_emprestimo, numero_parcelas):
        self.valor_emprestimo = valor_emprestimo
        self.numero_parcelas = numero_parcelas

    def calcular_parcela(self):
        # Cálculo simples sem juros compostos apenas para validação inicial
        return self.valor_emprestimo / self.numero_parcelas

class AnaliseRisco:
    def analisar(self, cliente, proposta):
        # 1. Validação de Idade
        if cliente.idade < 18:
            return {"status": "Reprovado", "motivo": "Menor de idade"}

        # 2. Validação de Restrição (Nome Sujo)
        if cliente.negativado:
            return {"status": "Reprovado", "motivo": "Cliente negativado no CPF"}

        # 3. Análise de Score (Define Taxa de Juros ou Reprovação)
        if cliente.score <= 300:
            return {"status": "Reprovado", "motivo": "Score muito baixo (Alto Risco)"}
        elif cliente.score <= 700:
            taxa_juros = 1.05 # 5%
            categoria = "Padrão"
        else:
            taxa_juros = 1.02 # 2%
            categoria = "Premium"

        # 4. Cálculo de Capacidade de Pagamento (Regra dos 30%)
        valor_parcela = proposta.calcular_parcela() * taxa_juros
        limite_parcela = cliente.renda * 0.30

        if valor_parcela > limite_parcela:
            return {
                "status": "Reprovado",
                "motivo": f"Parcela (R${valor_parcela:.2f}) excede 30% da renda"
            }

        # Se passou por tudo:
        return {
            "status": "Aprovado",
            "categoria": categoria,
            "taxa_juros": f"{(taxa_juros-1)*100:.0f}%",
            "valor_parcela_final": f"R$ {valor_parcela:.2f}"
        }

# --- TESTE DA SOLUÇÃO ---
# Criando cenários para validar
cliente1 = Cliente("João Estudante", 20, 2500, 750, False) # Bom pagador
cliente2 = Cliente("Maria Devedora", 35, 5000, 250, True)  # Negativada
cliente3 = Cliente("Pedro Arriscado", 40, 3000, 400, False) # Score médio, tenta empréstimo alto

proposta1 = Proposta(5000, 12) # Empréstimo aceitável
proposta2 = Proposta(50000, 24) # Empréstimo alto demais para Pedro

sistema = AnaliseRisco()

print(f"Análise João: {sistema.analisar(cliente1, proposta1)}")
print(f"Análise Maria: {sistema.analisar(cliente2, proposta1)}")
print(f"Análise Pedro: {sistema.analisar(cliente3, proposta2)}")

Análise João: {'status': 'Aprovado', 'categoria': 'Premium', 'taxa_juros': '2%', 'valor_parcela_final': 'R$ 425.00'}
Análise Maria: {'status': 'Reprovado', 'motivo': 'Cliente negativado no CPF'}
Análise Pedro: {'status': 'Reprovado', 'motivo': 'Parcela (R$2187.50) excede 30% da renda'}


In [ ]:
%%writefile regras_credito.pl

% --- BASE DE FATOS (Simulando o Banco de Dados) ---
% cliente(Nome, Idade, Renda, Score, Negativado).
% Negativado: sim (nome sujo) ou nao (nome limpo).

cliente(joao, 20, 2500, 750, nao).
cliente(maria, 35, 5000, 250, sim).
cliente(pedro, 40, 3000, 400, nao).
cliente(ana_menor, 17, 1000, 800, nao).

% --- REGRAS DO SISTEMA ---

% 1. Regra de Maioridade
maior_idade(Pessoa) :-
    cliente(Pessoa, Idade, _, _, _),
    Idade >= 18.

% 2. Regra do Nome Limpo (Compliance)
nome_limpo(Pessoa) :-
    cliente(Pessoa, _, _, _, nao).

% 3. Regra do Score e Taxa de juros
%Define a taxa baseada na faixa de score
taxa_juros(Pessoa, 1.02) :-
    cliente(Pessoa, _, _, Score, _),
    Score > 700.

taxa_juros(Pessoa, 1.05) :-
    cliente(Pessoa, _, _, Score, _),
    Score >= 301,
    Score =< 700.

% Se o score for <= 300, não há regra de taxa, o que causará falha (reprovação).

% 4. Regra de Capacidade de Pagamento (30% da Renda)
% Verifica se a Parcela cabe no bolso
capacidade_pagamento(Pessoa, ValorParcela) :-
    cliente(Pessoa, _, Renda, _, _),
    Limite is Renda * 0.3,
    ValorParcela =< Limite.

% --- MOTOR DE DECISÃO (Regra Final) ---

% Regra para EMPRÉSTIMO APROVADO
% Onde o Prolog tenta satisfazer todas as condições simultaneamente.

analise_credito(Pessoa, ValorEmprestimo, NumParcelas, 'APROVADO') :-
    maior_idade(Pessoa),
    nome_limpo(Pessoa),
    taxa_juros(Pessoa, Taxa),
    ValorParcela is (ValorEmprestimo / NumParcelas) * Taxa,
    capacidade_pagamento(Pessoa, ValorParcela).

% Regras para EMPRÉSTIMO REPROVADO (Diagnóstico de Falhas)
% Útil para dizer o "motivo" no relatório.

analise_credito(Pessoa, _, _, 'REPROVADO: Menor de Idade') :-
    \+ maior_idade(Pessoa).

analise_credito(Pessoa, _, _, 'REPROVADO: Cliente Negativado') :-
    maior_idade(Pessoa),
    \+ nome_limpo(Pessoa).

analise_credito(Pessoa, _, _, 'REPROVADO: Score Baixo (Alto Risco)') :-
    maior_idade(Pessoa),
    nome_limpo(Pessoa),
    \+ taxa_juros(Pessoa, _).

analise_credito(Pessoa, ValorEmprestimo, NumParcelas, 'REPROVADO: Renda Insuficiente') :-
    maior_idade(Pessoa),
    nome_limpo(Pessoa),
    taxa_juros(Pessoa, Taxa),
    ValorParcela is (ValorEmprestimo / NumParcelas) * Taxa,
    \+ capacidade_pagamento(Pessoa, ValorParcela).

Overwriting regras_credito.pl


In [ ]:
# !swipl para consultar nosso arquivo

queries = [
    "analise_credito(joao, 5000, 12, Resultado)", # Caso Ideal
    "analise_credito(maria, 5000, 12, Resultado)",  # Caso Negativada
    "analise_credito(pedro, 50000, 24, Resultado)",  # Caso Renda Insuficiente
    "analise_credito(ana_menor, 1000, 10, Resultado)"  # Caso Menor de Idade
]

print("-" * 40)
print("RESULTADOS DA ANÁLISE LÓGICA (PROLOG)")
print("-" * 40)

for query in queries:
  # Executa o SWI-Prolog via linha do comando
  # -s carrega o arquivo
  # -g executa a query e imprime
  # -t halt encerra o processo
  command = f"swipl -s regras_credito.pl -g \"{query}, write(Resultado), nl.\" -t halt"
  print(f"Consultando: {query}")
  !{command}
  print("-" * 40)

----------------------------------------
RESULTADOS DA ANÁLISE LÓGICA (PROLOG)
----------------------------------------
Consultando: analise_credito(joao, 5000, 12, Resultado)
APROVADO
----------------------------------------
Consultando: analise_credito(maria, 5000, 12, Resultado)
REPROVADO: Cliente Negativado
----------------------------------------
Consultando: analise_credito(pedro, 50000, 24, Resultado)
REPROVADO: Renda Insuficiente
----------------------------------------
Consultando: analise_credito(ana_menor, 1000, 10, Resultado)
REPROVADO: Menor de Idade
----------------------------------------


In [27]:
%%writefile analise_funcional.lisp

;; --- DEFINIÇÃO DOS DADOS (Listas de Propriedades) ---
;; Em Lisp, usamos listas para tudo. Aqui, cada cliente é uma lista de pares.
;; nil = falso (nome limpo), t = verdadeiro (nome sujo)
(defparameter *joao* '((:nome "Joao") (:idade 20) (:renda 2500) (:score 750) (:negativado nil)))
(defparameter *maria* '((:nome "Maria") (:idade 35) (:renda 5000) (:score 250) (:negativado t)))
(defparameter *pedro* '((:nome "Pedro") (:idade 40) (:renda 3000) (:score 400) (:negativado nil)))
(defparameter *ana* '((:nome "Ana")  (:idade 17) (:renda 1000) (:score 800) (:negativado nil)))

;; Função auxiliar para pegar valor da lista (como se fosse um dicionário)
(defun get-val (chave lista)
  (second (assoc chave lista)))

;; --- FUNÇÕES PURAS (O Pipeline de Validação) ---

;; 1. Valida Idade
(defun validar-idade (cliente)
  (if (>= (get-val :idade cliente) 18)
      (list :ok cliente) ;; Retorna uma lista com status OK e os dados
      (list :erro "Reprovado: Menor de Idade")))

;; 2. Valida Nome Limpo (Compliance)
(defun validar-nome (resultado-anterior)
  (let ((status (first resultado-anterior))
        (dados (second resultado-anterior)))
    ;; Se já deu erro na função anterior, apenas repassa o erro
    (if (eq status :erro)
        resultado-anterior
        ;; Senão, verifica se está negativado
        (if (not (get-val :negativado dados))
            (list :ok dados)
            (list :erro "Reprovado: Cliente Negativado")))))

;; 3. Calcula e Valida Capacidade Financeira
(defun validar-financeiro (resultado-anterior valor-emprestimo num-parcelas)
  (let ((status (first resultado-anterior))
        (dados (second resultado-anterior)))
    (if (eq status :erro)
        resultado-anterior ;; Repassa erro anterior
        (let* ((score (get-val :score dados))
               (renda (get-val :renda dados))
               ;; Define Juros baseado no Score (Logica Condicional)
               (taxa (cond ((<= score 300) 0)     ;; Score muito baixo
                           ((<= score 700) 1.05)  ;; Padrao (5%)
                           (t 1.02)))             ;; Premium (2%)
               (parcela (* (/ valor-emprestimo num-parcelas) taxa))
               (limite (* renda 0.30)))

          ;; Decisão Final
          (cond
            ((= taxa 0) (list :erro "Reprovado: Score Baixo (Alto Risco)"))
            ((> parcela limite) (list :erro "Reprovado: Renda Insuficiente para a parcela"))
            (t (list :aprovado (format nil "APROVADO! Parcela: R$~,2f (Juros: ~d%)"
                                           parcela (round (* (- taxa 1) 100))))))))))

;; --- FUNÇÃO ORQUESTRADORA ---
;; Encadeia as funções: Idade -> Nome -> Financeiro
(defun processar-analise (cliente valor parcelas)
  (format t "~%Analisando cliente: ~a... " (get-val :nome cliente))
  (print
    (validar-financeiro
      (validar-nome
        (validar-idade cliente))
      valor parcelas)))

;; --- EXECUÇÃO DOS TESTES ---
(format t "~%--- INICIANDO SISTEMA BANCARIO (LISP) ---~%")

;; Teste 1: Joao (Deve Aprovar)
(processar-analise *joao* 5000 12)

;; Teste 2: Maria (Deve Reprovar - Negativada)
(processar-analise *maria* 5000 12)

;; Teste 3: Pedro (Deve Reprovar - Renda Insuficiente para 50k)
(processar-analise *pedro* 50000 24)

;; Teste 4: Ana (Deve Reprovar - Menor de Idade)
(processar-analise *ana* 1000 10)

Writing analise_funcional.lisp


In [28]:
# Executa o interpretador SBCL (Steel Bank Common Lisp)
!sbcl --noinform --non-interactive --load analise_funcional.lisp --quit


--- INICIANDO SISTEMA BANCARIO (LISP) ---

Analisando cliente: Joao... 
(:APROVADO "APROVADO! Parcela: R$425.00 (Juros: 2%)") 
Analisando cliente: Maria... 
(:ERRO "Reprovado: Cliente Negativado") 
Analisando cliente: Pedro... 
(:ERRO "Reprovado: Renda Insuficiente para a parcela") 
Analisando cliente: Ana... 
(:ERRO "Reprovado: Menor de Idade") 